<a href="https://colab.research.google.com/github/akalpit23/Interpretable-ML/blob/main/Assignment_04_Interpretable_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #04



## Akalpit Dawkhar

### **Interpretable ML-2**

### Overview

In [2]:
pip install --upgrade skope-rules

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import six
from imodels import  RuleFitRegressor
from rulefit import RuleFit
from skrules import SkopeRules
from skrules import SkopeRules as SkopeRulesClassifier

In [13]:
# Load the dataset
credit_card_data = pd.read_csv('e_waste_dataset_with_profit.csv')

# Print the shape of the dataset
print(credit_card_data.shape)

# Print the first few rows of the dataset
print(credit_card_data.head())


(10000, 12)
              Item Category  Gold  Silver  Platinum  Rhodium  Nickel   Tin  \
0        iPhone 11     Cat3  3.58    2.95      1.73     8.92    1.91  1.01   
1          Toaster     Cat2  7.21    4.31      6.21     5.63    9.59  7.65   
2          Speaker     Cat4  8.91    5.09      2.42     7.70    1.09  1.49   
3   Microwave Oven     Cat2  2.62    3.84      2.98     7.66    9.41  2.25   
4  Air Conditioner     Cat1  3.47    3.89      6.20     4.35    5.07  8.65   

   Lithium  Aluminum  Carbon  Profit ($)  
0     1.82      1.27    9.51      270.34  
1     0.51      3.03    4.22      689.75  
2     7.42      3.63    8.83      570.43  
3     7.84      6.18    6.36      290.78  
4     8.62      0.82    5.53      505.16  


In [16]:
# Load your data
X = credit_card_data.drop(['Category', 'Item', 'Profit ($)'], axis=1) # Drop the target variable
y = credit_card_data['Profit ($)'] # Keep only the target variable
# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Check the shape of your DataFrame
print(X_train.shape)

# Check the column names of your DataFrame
print(X_train.columns)

(8000, 9)
Index(['Gold', 'Silver', 'Platinum', 'Rhodium', 'Nickel', 'Tin', 'Lithium',
       'Aluminum', 'Carbon'],
      dtype='object')


In [18]:
# Initialize the RuleFitClassifier
rf = RuleFitRegressor()

# Fit the model to your data
rf.fit(X_train, y_train)

# Use the model to make predictions
predictions = rf.predict(X_test)

In [19]:
rule_df = rf.visualize()
rule_df

,rule,coef
0,Gold,49.25
1,Silver,0.50
2,Platinum,53.71


In [20]:
# Calculate the mean absolute error
mae = mean_absolute_error(y_test, predictions)

# Calculate the mean squared error
mse = mean_squared_error(y_test, predictions)

# Calculate the R-squared score
r2 = r2_score(y_test, predictions)

# Print the metrics
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2 Score: {r2}')

MAE: 1.061747896385718
MSE: 5.91225880952765
R2 Score: 0.9998685273117945


In [25]:
feature_names = credit_card_data.drop('Profit ($)', axis=1).columns.tolist()


clf = SkopeRulesClassifier(max_depth_duplication=None,
                 n_estimators=30,
                 precision_min=0.2,
                 recall_min=0.01,
                 feature_names=feature_names)

X, y = credit_card_data.data, credit_card_data.target > 25
X_train, y_train = X[:len(y)//2], y[:len(y)//2]
X_test, y_test = X[len(y)//2:], y[len(y)//2:]
clf.fit(X_train, y_train)
y_score = clf._score_top_rules(X_test) # Get a risk score for each test example
precision, recall, _ = precision_recall_curve(y_test, y_score)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall curve')
plt.show()


AttributeError: 'DataFrame' object has no attribute 'data'